## MEAM 5460 Project 1: Blade Element Momentum Theory
### Author: Nathaniel Ruhl

### Introduction

In this report, I expand the BEMT tool first developed in Homework 3. We are still consider a rotor in hover, but we consider linear variations of rotor properties such as rotor twist distribution $\theta(r)$, blade section chord length $c(r)$, and linear airfoil lift-curve slopes $c_{l,\alpha}$. The enumerated list below shows the key assumptions and models built into the "BEMT2" tool, which can be directly compared to the BEMT tool created for Homework 3:

1. The rotor is in hover: $\lambda_c = 0$
2. Linear twist distribution, specified by input collective at 75%R and the linear twist rate: $\theta(r) = \theta_{75} + \theta_{tw}(r-0.75)$
3. Linear taper of local blade chord, specified by the chord at 75%R and the taper rate (TR): $c(r) = c_{75}+\frac{1}{TR}(r-0.75)$
4. Linear airfoil lift-curve slopes, specified by the slope at 75%R and the airfoil varitation root-to-tip (AV): $c_{l,\alpha}(r)=c_{l,\alpha}^{(75)}+AV(r-0.75)$
5. The Prandtl tip loss factor, $0<F<1$, describes the increase of induced velocity near the rotor tip

### The "Rotor2" class and baseline test rotor specifications

The test rotor specifications that will be considered in this report are used as the default instance properties in the "Rotor2" class. The default properties are

- $N_b$ = 1
- $R$ = 30 ft
- $v_{tip}$ = 780 ft/sec

and the (property) coefficients defining the linear variations are

__Twist distribution__:
- $\theta_{75}=5^\circ$ (I define this because the assignment does not specify)
- $\theta_{tw}=-8^\circ$ per non-dimensional radius

__Taper__:
- $c_{75} = 2$ ft
- $TR = 2$ ft per non-dimensional radius

__Airfoil variation__:
- $c_{l,\alpha}^{(75)}=5\pi$
- $AV=4\pi$ per non-dimensional radius







### BEMT Equations with Prandtl Tip Loss Factor

### Analysis